In [1]:
import findspark
findspark.init("/home/ubuntu/spark-2.1.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("yye090_722").getOrCreate()

In [2]:
df = spark.read.load("./final1.csv",format ="csv",header="true")

In [3]:
from pyspark.sql import *
df.printSchema()
df = df.withColumn("year", df["year"].cast("integer")).withColumn("month", df["month"].cast("integer"))\
.withColumn("hour", df["hour"].cast("integer"))\
.withColumn("day", df["day"].cast("integer"))\
.withColumn("SO2", df["SO2"].cast("float"))\
.withColumn("NO2", df["NO2"].cast("float"))\
.withColumn("CO", df["CO"].cast("float"))\
.withColumn("O3", df["O3"].cast("float"))\
.withColumn("TEMP", df["TEMP"].cast("float"))\
.withColumn("WSPM", df["WSPM"].cast("float"))
df.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- WSPM: string (nullable = true)
 |-- station: string (nullable = true)
 |-- PM25: string (nullable = true)
 |-- SO2: string (nullable = true)
 |-- NO2: string (nullable = true)
 |-- CO: string (nullable = true)
 |-- O3: string (nullable = true)
 |-- TEMP: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- PM_Rating: string (nullable = true)

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- WSPM: float (nullable = true)
 |-- station: string (nullable = true)
 |-- PM25: string (nullable = true)
 |-- SO2: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- O3: float (nullable = true)
 |-- TEMP: float (nullable = true)
 |-- Season: string (nullable = true)
 |-- PM_Rating: 

In [4]:
from pyspark.ml.feature import Normalizer, VectorAssembler, StandardScaler, StringIndexer
cat_column = ["station","Season","PM_Rating"]
for i in cat_column:
    indexer = StringIndexer(inputCol=i, outputCol=i + "Index")
    model = indexer.fit(df)
    new = model.transform(df)
    df = new
df.show()
df = df.drop(*["PM25"])

+----+-----+---+----+----+------------+----+----+----+-----+----+----+------+---------+------------+-----------+--------------+
|year|month|day|hour|WSPM|     station|PM25| SO2| NO2|   CO|  O3|TEMP|Season|PM_Rating|stationIndex|SeasonIndex|PM_RatingIndex|
+----+-----+---+----+----+------------+----+----+----+-----+----+----+------+---------+------------+-----------+--------------+
|2013|    3|  1|   0| 5.7|Nongzhanguan| 5.0| 4.0|12.0|200.0|85.0|-0.5|Spring|Excellent|         7.0|        2.0|           2.0|
|2013|    3|  1|   1| 3.9|Nongzhanguan| 8.0| 6.0|14.0|200.0|84.0|-0.7|Spring|Excellent|         7.0|        2.0|           2.0|
|2013|    3|  1|   2| 5.3|Nongzhanguan| 3.0| 5.0|14.0|200.0|83.0|-1.2|Spring|Excellent|         7.0|        2.0|           2.0|
|2013|    3|  1|   3| 4.9|Nongzhanguan| 5.0| 5.0|14.0|200.0|84.0|-1.4|Spring|Excellent|         7.0|        2.0|           2.0|
|2013|    3|  1|   4| 3.2|Nongzhanguan| 5.0| 6.0|21.0|200.0|77.0|-1.9|Spring|Excellent|         7.0|    

In [5]:
from pyspark.sql import *
new_df = df.drop(*["station","Season","PM_Rating"])
new_df = new_df.withColumn("year", new_df["year"].cast("integer")).withColumn("month", new_df["month"].cast("integer"))\
.withColumn("hour", new_df["hour"].cast("integer"))\
.withColumn("day", new_df["day"].cast("integer"))\
.withColumn("SO2", new_df["SO2"].cast("float"))\
.withColumn("NO2", new_df["NO2"].cast("float"))\
.withColumn("CO", new_df["CO"].cast("float"))\
.withColumn("O3", new_df["O3"].cast("float"))\
.withColumn("TEMP", new_df["TEMP"].cast("float"))\
.withColumn("WSPM", new_df["WSPM"].cast("float"))
new_df.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- WSPM: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- O3: float (nullable = true)
 |-- TEMP: float (nullable = true)
 |-- stationIndex: double (nullable = true)
 |-- SeasonIndex: double (nullable = true)
 |-- PM_RatingIndex: double (nullable = true)



In [6]:
from pyspark.ml.feature import RFormula
f = RFormula(formula="PM_RatingIndex~.",featuresCol="features",labelCol="label")
output = f.fit(new_df).transform(new_df)
feature_selection = output.select("features","label")
feature_selection.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2013.0,3.0,1.0,0...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,2...|  2.0|
|[2013.0,3.0,1.0,3...|  2.0|
|[2013.0,3.0,1.0,4...|  2.0|
|[2013.0,3.0,1.0,5...|  2.0|
|[2013.0,3.0,1.0,6...|  2.0|
|[2013.0,3.0,1.0,7...|  2.0|
|[2013.0,3.0,1.0,8...|  2.0|
|[2013.0,3.0,1.0,9...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
|[2013.0,3.0,1.0,1...|  2.0|
+--------------------+-----+
only showing top 20 rows



In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
(trainingData, testData) = feature_selection.randomSplit([0.7, 0.3])

In [8]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [9]:
model = dt.fit(trainingData)

In [10]:
prediction = model.transform(testData)
predfinal = prediction.select("features","prediction","probability","label")
correcPrediction = predfinal.filter(predfinal.prediction ==predfinal.label).count()
totaldata = predfinal.count()
print("Correct prediction: " + str(correcPrediction) + " total data: " + str(totaldata) + " accuracy: " +\
      str(correcPrediction/totaldata))

Correct prediction: 37798 total data: 64696 accuracy: 0.584240138493879


In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'label')
print("DTC")
print(my_binary_eval.evaluate(prediction))

DTC
0.731642977034116


In [12]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rfc_acc = acc_evaluator.evaluate(prediction)
print('A decision tree has an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print("Test Error = %g" % (1.0 - rfc_acc))


A decision tree has an accuracy of: 58.42%
Test Error = 0.41576


In [13]:
print(model.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_483f9caa32a5d4eb0db5) of depth 5 with 63 nodes
  If (feature 7 <= 700.0)
   If (feature 7 <= 400.0)
    If (feature 11 in {0.0,2.0})
     If (feature 9 <= 8.100000381469727)
      If (feature 1 <= 3.0)
       Predict: 2.0
      Else (feature 1 > 3.0)
       Predict: 2.0
     Else (feature 9 > 8.100000381469727)
      If (feature 6 <= 15.0)
       Predict: 2.0
      Else (feature 6 > 15.0)
       Predict: 1.0
    Else (feature 11 not in {0.0,2.0})
     If (feature 7 <= 300.0)
      If (feature 6 <= 13.0)
       Predict: 2.0
      Else (feature 6 > 13.0)
       Predict: 2.0
     Else (feature 7 > 300.0)
      If (feature 9 <= 2.5999999046325684)
       Predict: 2.0
      Else (feature 9 > 2.5999999046325684)
       Predict: 1.0
   Else (feature 7 > 400.0)
    If (feature 11 in {2.0})
     If (feature 9 <= 14.899999618530273)
      If (feature 7 <= 500.0)
       Predict: 1.0
      Else (feature 7 > 500.0)
       Predict: 3.0
    

In [21]:
print(new_df.columns)

['year', 'month', 'day', 'hour', 'WSPM', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'stationIndex', 'SeasonIndex', 'PM_RatingIndex']
